In [1]:
import math
import pandas as pd
import numpy as np

In [57]:
df = pd.read_csv('data\property_data.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.shape

(1604, 26)

In [58]:
# delete columns [id, assessor id, legal description, subdivision, owner, owner occupied]
# move last sale price and last sale date to the last two columns


In [59]:
df = df.drop(['id', 'assessorID', 'legalDescription', 'subdivision', 'owner', 'ownerOccupied','formattedAddress','addressLine1','addressLine2'], axis=1)
df.rename(columns = {'squareFootage':'Sq_Ft'}, inplace = True)
#df['lastSaleDate'] = pd.to_datetime(df.lastSaleDate)
#df['lastSaleDate'] = df['lastSaleDate'].dt.strftime('%Y/%m/%d')

In [60]:
df["features"] = df["features"].apply(lambda x : dict(eval(x)) )
df2 = df["features"].apply(pd.Series )


In [61]:
df3 = pd.concat([df, df2], axis=1).drop('features', axis=1)
df3

,city,state,zipCode,bedrooms,county,Sq_Ft,yearBuilt,bathrooms,lotSize,propertyType,...,heating,heatingType,unitCount,architectureType,foundationType,roofType,roomCount,pool,poolType,viewType
0,Lithonia,GA,30058,3.0,DeKalb County,2432.0,2003.0,2.5,13068.0,Single Family,...,True,Central,1.0,Traditional,NaN,NaN,NaN,NaN,NaN,NaN
1,Lithonia,GA,30058,3.0,Dekalb,2006.0,2000.0,3.0,4356.0,Single Family,...,True,Central,1.0,Traditional,Concrete,NaN,NaN,NaN,NaN,NaN
2,Lithonia,GA,30058,3.0,Dekalb,2036.0,1997.0,3.0,13068.0,Single Family,...,True,Central,1.0,Split Level,Concrete,NaN,NaN,NaN,NaN,NaN
3,Lithonia,GA,30058,4.0,Dekalb,2720.0,2002.0,4.0,17424.0,Single Family,...,True,Central,1.0,Traditional,Concrete,NaN,NaN,NaN,NaN,NaN
4,Lithonia,GA,30058,4.0,Dekalb,2530.0,1997.0,3.0,17424.0,Single Family,...,True,Central,1.0,Traditional,Concrete,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,Decatur,GA,30035,3.0,Dekalb,1687.0,1969.0,3.0,13068.0,Single Family,...,True,Central,1.0,Split Level,Concrete,Asphalt,6.0,NaN,NaN,NaN
1600,Decatur,GA,30034,3.0,DeKalb County,1232.0,1963.0,2.0,13068.0,Single Family,...,True,Central,1.0,Ranch / Rambler,Concrete,Asphalt,5.0,NaN,NaN,NaN
1601,Newnan,GA,30265,4.0,Coweta,2421.0,2018.0,3.0,NaN,Single Family,...,True,Central,NaN,NaN,Slab,Asbestos,NaN,NaN,NaN,NaN
1602,Peachtree City,GA,30269,3.0,Fayette County,1826.0,1998.0,2.5,NaN,Single Family,...,True,Central,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
#df3["taxAssessment"].apply(pd.Series )
df["taxAssessment"] = df["taxAssessment"].apply(lambda x : dict(eval(x)) )
df4 = df["taxAssessment"].apply(pd.Series )

#df4['2019'] = df4['2019'].apply(lambda x: sum(x.values()))
#df4

In [63]:
df4['2019']=df4['2019'].apply(lambda x: sum(x.values()) if isinstance(x, dict) else x )
df4['2020']=df4['2020'].apply(lambda x: sum(x.values()) if isinstance(x, dict) else x )
df4['2021']=df4['2021'].apply(lambda x: sum(x.values()) if isinstance(x, dict) else x )
df4['2017']=df4['2017'].apply(lambda x: sum(x.values()) if isinstance(x, dict) else x )
df4['2018']=df4['2018'].apply(lambda x: sum(x.values()) if isinstance(x, dict) else x )

In [64]:
df4= df4[['2017', '2018', '2019','2020','2021']]


In [65]:
try:
    z= df4['2020'] / df4['2019']
except (ZeroDivisionError, ValueError):
    print(0) 
    
print(type(z))
new_array = z.to_numpy() 
mean_1= np.nanmean(new_array)
print(mean_1)#mean of parameter from 2019 to 2020

<class 'pandas.core.series.Series'>
1.1544281837852004


In [66]:
df4['2020']=df4['2020'].fillna(df4['2019']* mean_1)
df4['2019']=df4['2019'].fillna(df4['2020']/ mean_1)


In [67]:
try:
    y= df4['2021'] / df4['2020']
except (ZeroDivisionError, ValueError):
    print(0) 
    

new_array = y.to_numpy() 
mean_2= np.nanmean(new_array)
print(mean_2)#mean of parameter from 2020 to 2021

1.164140430852898


In [68]:
df4['2021']=df4['2021'].fillna(df4['2020']* mean_2)
df4['2020']=df4['2020'].fillna(df4['2021']/ mean_2)
df4['2019']=df4['2019'].fillna(df4['2020']/ mean_1)
df4

,2017,2018,2019,2020,2021
0,NaN,NaN,138000.000000,137280.000000,159813.198347
1,NaN,NaN,84000.000000,96971.967438,125440.000000
2,NaN,NaN,104160.000000,109840.000000,125120.000000
3,NaN,NaN,144880.000000,167253.555267,175440.000000
4,NaN,NaN,117600.000000,124480.000000,144912.200833
...,...,...,...,...,...
1599,NaN,NaN,101196.761589,116824.393686,136000.000000
1600,NaN,NaN,75911.175100,87634.000000,102018.282517
1601,NaN,NaN,192136.681273,221808.000000,221808.000000
1602,NaN,NaN,250480.000000,289161.171475,336624.210746


In [69]:
df5 = pd.concat([df3, df4], axis=1)
df5 = df5.drop(['taxAssessment','propertyTaxes','city','state','county','lastSaleDate'], axis=1)

In [70]:
df6 = pd.get_dummies(df5, prefix='type', prefix_sep='.', 
                            columns=['propertyType','coolingType', 'exteriorType','fireplaceType','garageType','heatingType','architectureType','foundationType','roofType','poolType','viewType'])
df6

,zipCode,bedrooms,Sq_Ft,yearBuilt,bathrooms,lotSize,lastSalePrice,longitude,latitude,cooling,...,type.Shingle (Not Wood),type.Steel,type.Wood Shake / Shingle,type.Wood Shingle,type.In-Ground Vinyl Pool,type.Pool and Hot Tub / Spa,"type.Pool, Concrete","type.Pool, Fiberglass","type.Pool, Gunite","type.Corner, Waterfront"
0,30058,3.0,2432.0,2003.0,2.5,13068.0,160000.0,-84.067653,33.723977,True,...,0,0,0,0,0,0,0,0,0,0
1,30058,3.0,2006.0,2000.0,3.0,4356.0,105000.0,-84.130286,33.719805,True,...,0,0,0,0,0,0,0,0,0,0
2,30058,3.0,2036.0,1997.0,3.0,13068.0,11569148.0,-84.160979,33.726402,True,...,0,0,0,0,0,0,0,0,0,0
3,30058,4.0,2720.0,2002.0,4.0,17424.0,100500.0,-84.067283,33.728890,True,...,0,0,0,0,0,0,0,0,0,0
4,30058,4.0,2530.0,1997.0,3.0,17424.0,123700.0,-84.160283,33.722459,True,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,30035,3.0,1687.0,1969.0,3.0,13068.0,90500.0,-84.219227,33.721101,True,...,0,0,0,0,0,0,0,0,0,0
1600,30034,3.0,1232.0,1963.0,2.0,13068.0,35000.0,-84.298741,33.712506,True,...,0,0,0,0,0,0,0,0,0,0
1601,30265,4.0,2421.0,2018.0,3.0,NaN,239900.0,-84.747476,33.366311,True,...,0,0,0,0,0,0,0,0,0,0
1602,30269,3.0,1826.0,1998.0,2.5,NaN,260000.0,-84.601763,33.417578,True,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df6= pd.get_dummies(df6
               ,columns = ['cooling']
               ,prefix = 'cooling'
               )
df6= pd.get_dummies(df6
               ,columns = ['fireplace']
               ,prefix = 'fireplace'
               )
df6= pd.get_dummies(df6
               ,columns = ['garage']
               ,prefix = 'garage'
               )
df6= pd.get_dummies(df6
               ,columns = ['heating']
               ,prefix = 'heating'
               )
df6= pd.get_dummies(df6
               ,columns = ['pool']
               ,prefix = 'pool'
               )
df6

,zipCode,bedrooms,Sq_Ft,yearBuilt,bathrooms,lotSize,lastSalePrice,longitude,latitude,floorCount,...,type.Pool and Hot Tub / Spa,"type.Pool, Concrete","type.Pool, Fiberglass","type.Pool, Gunite","type.Corner, Waterfront",cooling_True,fireplace_True,garage_True,heating_True,pool_True
0,30058,3.0,2432.0,2003.0,2.5,13068.0,160000.0,-84.067653,33.723977,2.0,...,0,0,0,0,0,1,1,1,1,0
1,30058,3.0,2006.0,2000.0,3.0,4356.0,105000.0,-84.130286,33.719805,2.0,...,0,0,0,0,0,1,1,1,1,0
2,30058,3.0,2036.0,1997.0,3.0,13068.0,11569148.0,-84.160979,33.726402,2.0,...,0,0,0,0,0,1,1,1,1,0
3,30058,4.0,2720.0,2002.0,4.0,17424.0,100500.0,-84.067283,33.728890,2.0,...,0,0,0,0,0,1,1,1,1,0
4,30058,4.0,2530.0,1997.0,3.0,17424.0,123700.0,-84.160283,33.722459,2.0,...,0,0,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,30035,3.0,1687.0,1969.0,3.0,13068.0,90500.0,-84.219227,33.721101,2.0,...,0,0,0,0,0,1,1,1,1,0
1600,30034,3.0,1232.0,1963.0,2.0,13068.0,35000.0,-84.298741,33.712506,1.0,...,0,0,0,0,0,1,1,1,1,0
1601,30265,4.0,2421.0,2018.0,3.0,NaN,239900.0,-84.747476,33.366311,2.0,...,0,0,0,0,0,1,1,0,1,0
1602,30269,3.0,1826.0,1998.0,2.5,NaN,260000.0,-84.601763,33.417578,2.0,...,0,0,0,0,0,1,1,1,1,0


In [72]:
#'cooling','fireplace','garage','heating','pool'

In [73]:
cols = list(df6.columns.values)
cols

['zipCode',
 'bedrooms',
 'Sq_Ft',
 'yearBuilt',
 'bathrooms',
 'lotSize',
 'lastSalePrice',
 'longitude',
 'latitude',
 'floorCount',
 'garageSpaces',
 'unitCount',
 'roomCount',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 'type.Apartment',
 'type.Condo',
 'type.Duplex-Triplex',
 'type.Manufactured',
 'type.Multi Family',
 'type.Single Family',
 'type.Townhouse',
 'type.Vacant',
 'type.Central',
 'type.Commercial',
 'type.Evaporative',
 'type.Other',
 'type.Window',
 'type.Aluminum / Vinyl Siding',
 'type.Asbestos Shingle',
 'type.Block',
 'type.Brick',
 'type.Brick / Stone',
 'type.Brick Veneer',
 'type.Combination',
 'type.Composition',
 'type.Concrete',
 'type.Frame / Masonry',
 'type.Frame Brick',
 'type.Log',
 'type.Masonry',
 'type.Metal',
 'type.Other',
 'type.Rock, Stone',
 'type.Shingle',
 'type.Shingle (Not Wood)',
 'type.Siding',
 'type.Stone Veneer',
 'type.Stucco',
 'type.Vinyl',
 'type.Wood',
 'type.Wood Shingle',
 'type.Wood Siding',
 'type.2 Story',
 'type.Masonry',


In [74]:
df6=df6[['zipCode',
 'bedrooms',
 'Sq_Ft',
 'yearBuilt',
 'bathrooms',
 'lotSize',
 'longitude',
 'latitude',
 'floorCount',
 'garageSpaces',
 'unitCount',
 'roomCount',
 'type.Apartment',
 'type.Condo',
 'type.Duplex-Triplex',
 'type.Manufactured',
 'type.Multi Family',
 'type.Single Family',
 'type.Townhouse',
 'type.Vacant',
 'type.Central',
 'type.Commercial',
 'type.Evaporative',
 'type.Other',
 'type.Window',
 'type.Aluminum / Vinyl Siding',
 'type.Asbestos Shingle',
 'type.Block',
 'type.Brick',
 'type.Brick / Stone',
 'type.Brick Veneer',
 'type.Combination',
 'type.Composition',
 'type.Concrete',
 'type.Frame / Masonry',
 'type.Frame Brick',
 'type.Log',
 'type.Masonry',
 'type.Metal',
 'type.Other',
 'type.Rock, Stone',
 'type.Shingle',
 'type.Shingle (Not Wood)',
 'type.Siding',
 'type.Stone Veneer',
 'type.Stucco',
 'type.Vinyl',
 'type.Wood',
 'type.Wood Shingle',
 'type.Wood Siding',
 'type.2 Story',
 'type.Masonry',
 'type.Prefab',
 'type.Single',
 'type.Attached',
 'type.Attached Garage',
 'type.Basement',
 'type.Built-in Garage',
 'type.Carport',
 'type.Detached',
 'type.Detached Garage',
 'type.Garage',
 'type.Mixed',
 'type.Underground / Basement',
 'type.Central',
 'type.Electric',
 'type.Floor / Wall',
 'type.Floor Furnace',
 'type.Forced Air',
 'type.Gas',
 'type.Heat Pump',
 'type.Other',
 'type.Bungalow',
 'type.Colonial',
 'type.Condo / Apartment',
 'type.Conventional',
 'type.Duplex Single',
 'type.French Provincial',
 'type.Log Cabin / Rustic',
 'type.Modern',
 'type.Other',
 'type.Ranch',
 'type.Ranch / Rambler',
 'type.Split Entry',
 'type.Split Level',
 'type.Townhouse',
 'type.Traditional',
 'type.Two Family',
 'type.Block',
 'type.Concrete',
 'type.Concrete Block',
 'type.Crawl / Raised',
 'type.Footing',
 'type.Masonry',
 'type.Other',
 'type.Pier',
 'type.Retaining Wall',
 'type.Slab',
 'type.Slab / Mat / Raft',
 'type.Aluminum',
 'type.Asbestos',
 'type.Asphalt',
 'type.Built-up',
 'type.Built-up (Layered Asphalt)',
 'type.Composition Shingle',
 'type.Fiberglass',
 'type.Metal',
 'type.Other',
 'type.Shingle (Not Wood)',
 'type.Steel',
 'type.Wood Shake / Shingle',
 'type.Wood Shingle',
 'type.In-Ground Vinyl Pool',
 'type.Pool and Hot Tub / Spa',
 'type.Pool, Concrete',
 'type.Pool, Fiberglass',
 'type.Pool, Gunite',
 'type.Corner, Waterfront',
 'cooling_True',
 'fireplace_True',
 'garage_True',
 'heating_True',
 'pool_True',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
'lastSalePrice']]


In [75]:
# ['cooling', 'coolingType','exteriorType', 'fireplace', 'fireplaceType','floorCount','garage','garageSpaces', 'garageType', 'heating', 'heatingType', 'unitCount', 'architectureType', 'foundationType', 'roofType', 'roomCount', 'pool', 'poolType', 'viewType']
#above are features
#[ 'type.Apartment', 'type.Condo', 'type.Duplex-Triplex', 'type.Manufactured', 'type.Multi Family', 'type.Single Family', 'type.Townhouse', 'type.Vacant']
#above are property type

In [76]:
df6

,zipCode,bedrooms,Sq_Ft,yearBuilt,bathrooms,lotSize,longitude,latitude,floorCount,garageSpaces,...,fireplace_True,garage_True,heating_True,pool_True,2017,2018,2019,2020,2021,lastSalePrice
0,30058,3.0,2432.0,2003.0,2.5,13068.0,-84.067653,33.723977,2.0,2.0,...,1,1,1,0,NaN,NaN,138000.000000,137280.000000,159813.198347,160000.0
1,30058,3.0,2006.0,2000.0,3.0,4356.0,-84.130286,33.719805,2.0,1.0,...,1,1,1,0,NaN,NaN,84000.000000,96971.967438,125440.000000,105000.0
2,30058,3.0,2036.0,1997.0,3.0,13068.0,-84.160979,33.726402,2.0,2.0,...,1,1,1,0,NaN,NaN,104160.000000,109840.000000,125120.000000,11569148.0
3,30058,4.0,2720.0,2002.0,4.0,17424.0,-84.067283,33.728890,2.0,2.0,...,1,1,1,0,NaN,NaN,144880.000000,167253.555267,175440.000000,100500.0
4,30058,4.0,2530.0,1997.0,3.0,17424.0,-84.160283,33.722459,2.0,2.0,...,1,1,1,0,NaN,NaN,117600.000000,124480.000000,144912.200833,123700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,30035,3.0,1687.0,1969.0,3.0,13068.0,-84.219227,33.721101,2.0,2.0,...,1,1,1,0,NaN,NaN,101196.761589,116824.393686,136000.000000,90500.0
1600,30034,3.0,1232.0,1963.0,2.0,13068.0,-84.298741,33.712506,1.0,1.0,...,1,1,1,0,NaN,NaN,75911.175100,87634.000000,102018.282517,35000.0
1601,30265,4.0,2421.0,2018.0,3.0,NaN,-84.747476,33.366311,2.0,NaN,...,1,0,1,0,NaN,NaN,192136.681273,221808.000000,221808.000000,239900.0
1602,30269,3.0,1826.0,1998.0,2.5,NaN,-84.601763,33.417578,2.0,2.0,...,1,1,1,0,NaN,NaN,250480.000000,289161.171475,336624.210746,260000.0


In [77]:
df6 = df6.drop(['2017','2018','roomCount','garageSpaces','unitCount'], axis=1)
#these variables contains too many empty rows

In [78]:
df6=df6.dropna()

In [79]:
df6['prediction'] = df6[['2021', 'lastSalePrice']].values.max(1)
df6

,zipCode,bedrooms,Sq_Ft,yearBuilt,bathrooms,lotSize,longitude,latitude,floorCount,type.Apartment,...,cooling_True,fireplace_True,garage_True,heating_True,pool_True,2019,2020,2021,lastSalePrice,prediction
0,30058,3.0,2432.0,2003.0,2.5,13068.0,-84.067653,33.723977,2.0,0,...,1,1,1,1,0,138000.000000,137280.000000,159813.198347,160000.0,1.600000e+05
1,30058,3.0,2006.0,2000.0,3.0,4356.0,-84.130286,33.719805,2.0,0,...,1,1,1,1,0,84000.000000,96971.967438,125440.000000,105000.0,1.254400e+05
2,30058,3.0,2036.0,1997.0,3.0,13068.0,-84.160979,33.726402,2.0,0,...,1,1,1,1,0,104160.000000,109840.000000,125120.000000,11569148.0,1.156915e+07
3,30058,4.0,2720.0,2002.0,4.0,17424.0,-84.067283,33.728890,2.0,0,...,1,1,1,1,0,144880.000000,167253.555267,175440.000000,100500.0,1.754400e+05
4,30058,4.0,2530.0,1997.0,3.0,17424.0,-84.160283,33.722459,2.0,0,...,1,1,1,1,0,117600.000000,124480.000000,144912.200833,123700.0,1.449122e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,30032,2.0,1202.0,1954.0,1.0,13068.0,-84.261128,33.733336,1.0,0,...,1,1,1,1,0,76400.000000,88198.313241,102675.222377,49900.0,1.026752e+05
1598,30034,3.0,1836.0,1989.0,3.0,17424.0,-84.228328,33.683464,2.0,0,...,1,1,1,1,0,107089.984764,123627.696613,143920.000000,131800.0,1.439200e+05
1599,30035,3.0,1687.0,1969.0,3.0,13068.0,-84.219227,33.721101,2.0,0,...,1,1,1,1,0,101196.761589,116824.393686,136000.000000,90500.0,1.360000e+05
1600,30034,3.0,1232.0,1963.0,2.0,13068.0,-84.298741,33.712506,1.0,0,...,1,1,1,1,0,75911.175100,87634.000000,102018.282517,35000.0,1.020183e+05


In [81]:
df6.to_csv("data\clean_data.csv")